In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(30)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

import os
import shutil
directory_out = './file/2.part'
if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)
import re

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = =  = = =  = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = =  = = =  = = =

import json

# = = =  = = =  = = =

import numpy as np

# = = =  = = =  = = =

import html as html_lib

# = = =  = = =  = = =

directory_in = './file/1.part_number'
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
for file in list_file:

    # = = =  = = =  = = =

    print('Data Loading...')
    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                           header=0,
                                                                           dtype=str).fillna('')
    print('Loading Done !')
    print()

    df_input['No'] = df_input['No'].astype(int)

    crawler_remain = len(df_input)

    print(f'总数量：{crawler_remain}')
    print()

    # = = =  = = =  = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, crawler_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = =  = = =  = = =

        try:
            data = {'dopartsearch': '1',
                    'partsearch[partnum][partsearch_007]': crawler_series['Part Number'],
                    'partsearch[manufacturer][partsearch_007]': crawler_series['Manufacturer'],
                    'partsearch[parttype][partsearch_007]': crawler_series['Type Code'],
                    'partsearch[do][partsearch_007]': '1',
                    'func': 'sendparttabsearch',
                    'payload': '{}',
                    'api_json_request': '1'}

            crawler_retry = 0
            while True:
                crawler_retry += 1

                try:
                    url_check = crawler_series['Url']

                    resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                         data=data,
                                         headers=get_header(),
                                         proxies=get_proxy(),
                                         timeout=(5, 15))

                    if resp.status_code == 200:
                        dict_ = resp.json()

                        if 'redirect_to_url' not in dict_ and 'collected_javascript' in dict_:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = =  = = =  = = =

            soup = BeautifulSoup(dict_['searchnoderesults'], 'lxml')
            html = etree.HTML(str(soup))

            # = = =  = = =  = = =

            list_tbody = html.xpath('//tbody[contains(@id, "listingcontainer[__GIP__")]')

            # = = =  = = =  = = =

            list_dict = [json.loads(tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/input[contains(@id, "listing_data_essential[__GIP__")]/@value')[0]) for tbody in list_tbody]

            # = = =  = = =  = = =

            list_type_code, list_part_code = [dict_['parttype'] for dict_ in list_dict], [dict_['partkey'] for dict_ in list_dict]

            # = = =  = = =  = = =

            array_index_type_code = np.where(np.array(list_type_code) == crawler_series['Type Code'])[0]

            # = = =  = = =  = = =

            list_dict_2 = [json.loads(tbody.xpath('./tr[1]/td/input[contains(@id, "listing_data_supplemental[__GIP__")]/@value')[0]) for tbody in list_tbody]

            # = = =  = = =  = = =

            list_part_number, list_manufacturer, list_choose = [dict_['partnumber'] for dict_ in list_dict_2], [dict_['catalogname'] for dict_ in list_dict_2], [dict_['paramdesc'].strip() for dict_ in list_dict_2]

            # = = =  = = =  = = =

            array_index_part_number, array_index_manufacturer = np.where(np.array(list_part_number) == crawler_series['Part Number'])[0], np.where(np.array(list_manufacturer) == crawler_series['Manufacturer'])[0]

            # = = =  = = =  = = =

            array_index_part = np.intersect1d(np.intersect1d(array_index_type_code, array_index_part_number), array_index_manufacturer)

            # = = =  = = =  = = =

            if array_index_part.size == 1:
                index_part = array_index_part[0]
            else:
                raise

            # = = =  = = =  = = =

            list_span = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row-moreinfo-truck"]/span[@class="span-link-underline-remover"]')
            list_note = [etree.tostring(span).decode('utf-8') for span in list_span]
            list_note = [re.sub(r'<.*?>', '', note) for note in list_note]
            list_note = [re.sub(r' +', ' ', note.strip()) for note in list_note]
            list_note = [html_lib.unescape(note) for note in list_note]

            dict_note, i = {}, 0
            for note in list_note:
                if note.strip() != '':
                    dict_note[str(i)] = note
                    i += 1

            # = = =  = = =  = = =

            list_a = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]')
            info = list_a[0].xpath('./@href')[0].strip() if list_a else ''
            info = info if info.startswith('https://www.rockauto.com/en/moreinfo.php?') else f'https://www.rockauto.com/en/moreinfo.php?pk={list_part_code[index_part]}&pt={list_type_code[index_part]}&Lennon=1'

            # = = =  = = =  = = =

            dict_note_2, i, list_span = {}, 0, list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row"]/span[contains(@title, "Please make sure")]/parent::div[@class="listing-text-row"]/span[contains(@title, "Please make sure")]/span[@class="listing-footnote-text"]')
            for i, span in enumerate(list_span):
                note = etree.tostring(span).decode('utf-8')
                note = re.sub(r'<.*?>', '', note)
                note = re.sub(r' +', ' ', note.strip())
                note = html_lib.unescape(note)

                if note != '':
                    dict_note_2[str(i)] = note
                    i += 1

            # = = =  = = =  = = =

            list_list_text = [text.split(',') for td in list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]') for text in td.xpath('./span[contains(@title, "Replaces these Alternate")]/text()')]
            list_text = [text.strip() for list_text in list_list_text for text in list_text if text.strip() != '']
            oem = ';'.join(list(dict.fromkeys(list_text)))

            # = = =  = = =  = = =

            list_input = list_tbody[index_part].xpath('./tr[1]/td/div[contains(@class, "listing-image-space-occupy")]/input[contains(@id, "jsninlineimg[__GIP__")]')
            list_dict = [json.loads(value) for input_ in list_input for value in input_.xpath('./@value')]

            dict_src, i = {}, 0
            for dict_ in list_dict:
                list_src = dict_.get('Slots', [])
                for src in list_src:
                    dict_src[str(i)] = f'''https://www.rockauto.com{src['ImageData']['Full'].strip()}'''
                    i += 1

            # = = =  = = =  = = =

            list_span = list_tbody[index_part].xpath('./tr[1]/td[contains(@id, "listingtd[__GIP__") and contains(@id, "__][price]")]/span[contains(@id, "dprice[__GIP__") and contains(@id, "__][td]")]/span/span[contains(@id, "dprice[__GIP__") and contains(@id, "__][v]")]')
            list_price = [etree.tostring(span).decode('utf-8') for span in list_span]
            list_price = [re.sub(r'<.*?>', '', price) for price in list_price]
            list_price = [re.sub(r' +', ' ', price.strip()) for price in list_price]
            list_price = [html_lib.unescape(price) for price in list_price]
            dict_price = {'0': '; '.join(list_price)}

            # = = =  = = =  = = =

            list_td = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]')
            list_price = [etree.tostring(option).decode('utf-8') for td in list_td for option in td.xpath('./div/div[contains(@class, "listing-optionchoice-multiple-container")]/span[contains(@id, "ddrepl[optionchoice[__GIP__") and contains(@id, "__]][container]")]/span[@class="hide-if-js"]/select[contains(@id, "optionchoice[__GIP__") and contains(@id, "__]")]/option[@value!=""]')]
            list_price = [re.sub(r'<.*?>', '', price) for price in list_price]
            list_price = [re.sub(r' +', ' ', price.strip()) for price in list_price]
            list_price = [html_lib.unescape(price) for price in list_price]
            if list_price:
                dict_price['1'] = {}
                for i, price in enumerate(list_price):
                    dict_price['1'][str(i)] = price

            # = = =  = = =  = = =

            with semaphore_correct:
                list_dict_correct.append({'No': crawler_series['No'],
                                          'Group': crawler_series['Group'],
                                          'Type': crawler_series['Type'],
                                          'Manufacturer': crawler_series['Manufacturer'],
                                          'Part Number': crawler_series['Part Number'],
                                          'OEM': oem,
                                          'Picture': '',
                                          'Url': crawler_series['Url'],
                                          'Json_Src': json.dumps(dict_src),
                                          'Info': info,
                                          'Json_Note 1': json.dumps(dict_note),
                                          'Json_Note 2': json.dumps(dict_note_2),
                                          'Choose': list_choose[index_part],
                                          'Json_Price': json.dumps(dict_price),
                                          'Type Code': crawler_series['Type Code'],
                                          'Part Code': list_part_code[index_part],
                                          'JOIN_MPNTCPC': f'''{crawler_series['Manufacturer']};{crawler_series['Part Number']};{crawler_series['Type Code']};{list_part_code[index_part]}'''})

            # = = =  = = =  = = =

            crawler_status = True
        except:
            crawler_series['Check_Url'] = url_check

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = =  = = =  = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and crawler_remain == 1 or crawler_remain % 30 == 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{crawler_retry}次] - [剩余{crawler_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_check}\n''')

        with semaphore_remain:
            crawler_remain -= 1

    # = = =  = = =  = = =

    pool.map(crawler, df_input.iterrows())

    print('Data Outputting...')
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                                 ascending=[True],
                                                                 ignore_index=True)
        df_correct.to_excel(os.path.join(directory_out, f'''{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx'''), index=False)
    print('Output Finished !')
    print()
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('- Error Occurred -')
        print()

# = = =  = = =  = = =

list_file = [file for file in sorted(list(os.walk(directory_out))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_out, file))]
for file in list_file:
    os.rename(os.path.join(directory_out, file), os.path.join(directory_out, re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)))

# = = =  = = =  = = =

print('Done ~')

[True] - [请求2次] - [剩余29条] - [1（100.00%）] - [2024/12/14 16:29:36] - 895. https://www.rockauto.com/en/parts/moog,541001,wheel+bearing+&+hub,1636

[True] - [请求27次] - [剩余28条] - [1（100.00%）] - [2024/12/14 16:29:36] - 872. https://www.rockauto.com/en/parts/moog,515212,wheel+bearing+&+hub,1636

[True] - [请求3次] - [剩余27条] - [1（100.00%）] - [2024/12/14 16:29:37] - 884. https://www.rockauto.com/en/parts/moog,518517,wheel+bearing+&+hub,1636

[True] - [请求12次] - [剩余26条] - [1（100.00%）] - [2024/12/14 16:29:37] - 891. https://www.rockauto.com/en/parts/moog,520004,wheel+bearing+&+hub,1636

[True] - [请求7次] - [剩余25条] - [1（100.00%）] - [2024/12/14 16:29:38] - 889. https://www.rockauto.com/en/parts/moog,520001,wheel+bearing+&+hub,1636

[True] - [请求8次] - [剩余24条] - [1（100.00%）] - [2024/12/14 16:29:39] - 881. https://www.rockauto.com/en/parts/moog,518513,wheel+bearing+&+hub,1636

[True] - [请求2次] - [剩余23条] - [1（100.00%）] - [2024/12/14 16:29:42] - 900. https://www.rockauto.com/en/parts/moog,541008,wheel+bearing+&+